# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
%pip install azureml-train-automl-runtime==1.57.0 -q
%pip install --upgrade azureml-sdk[notebooks,automl] -q
%pip install azureml-widgets -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-sql 2024.5.0 requires dask[dataframe]>=2024.4.1, but you have dask 2023.2.0 which is incompatible.
dask-sql 2024.5.0 requires distributed>=2024.4.1, but you have distributed 2023.2.0 which is incompatible.
dask-sql 2024.5.0 requires pandas>=1.4.0, but you have pandas 1.3.5 which is incompatible.
dask-expr 1.1.14 requires dask==2024.9.0, but you have dask 2023.2.0 which is incompatible.
dask-expr 1.1.14 requires pandas>=2, but you have pandas 1.3.5 which is incompatible.
azureml-automl-dnn-nlp 1.57.0 requires torch==1.13.1, but you have torch 2.4.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pa

In [2]:


import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.57.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
session_id = "268930"
subscription_id= "9a7511b8-150f-4a58-8528-3e7d50216c31"
resource_group= f"aml-quickstarts-{session_id}"
workspace_name= f"quick-starts-ws-{session_id}"

ws = Workspace.get(name=workspace_name, subscription_id=subscription_id, resource_group=resource_group)
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'automl-exp'
project_folder = './capstone'
experiment=Experiment(ws, experiment_name)
experiment

Performing interactive authentication. Please follow the instructions on the terminal.
Interactive authentication successfully completed.
quick-starts-ws-268930
aml-quickstarts-268930
westeurope
9a7511b8-150f-4a58-8528-3e7d50216c31


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code GZ9BJRRTG to authenticate.


Name,Workspace,Report Page,Docs Page
automl-exp,quick-starts-ws-268930,Link to Azure Machine Learning studio,Link to Documentation


In [4]:


# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "heart-failure"
description_text = "Heart Failure Prediction Dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/hknguyen20/nd00333-capstone/refs/heads/master/heart.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)
df = dataset.to_pandas_dataframe()
df.head()


{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,False,Normal,172,False,0.0,Up,False
1,49,F,NAP,160,180,False,Normal,156,False,1.0,Flat,True
2,37,M,ATA,130,283,False,ST,98,False,0.0,Up,False
3,48,F,ASY,138,214,False,Normal,108,True,1.5,Flat,True
4,54,M,NAP,150,195,False,Normal,122,False,0.0,Up,False


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "compute-cluster-1"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_v2',# for GPU, use "STANDARD_NC6"
                                                           vm_priority = 'lowpriority', # optional
                                                           min_nodes=1,
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded......................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy',
    "enable_early_stopping": True,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             test_size=0.2,
                             label_column_name="HeartDisease",   
                             path = project_folder,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [7]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-exp,AutoML_8d854f6c-f80a-439b-abfb-6d9a63b2954b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

2024-10-21 08:54:55.491024: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-21 08:54:56.998975: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-21 08:54:57.440258: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-21 08:55:01.130337: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-21 08:55:08.490667: W tensorflow/comp

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
automl-exp,AutoML_8d854f6c-f80a-439b-abfb-6d9a63b2954b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number of training samples are fewer than 1000, and 3 folds in all other cases.
              Learn mo

{'runId': 'AutoML_8d854f6c-f80a-439b-abfb-6d9a63b2954b',
 'target': 'compute-cluster-1',
 'status': 'Completed',
 'startTimeUtc': '2024-10-21T08:54:33.643567Z',
 'endTimeUtc': '2024-10-21T09:20:26.202238Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'compute-cluster-1',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-exp","subscription_id":"9a7511b8-150f-4a58-8528-3e7d50216c31","resource_group":"aml-quickstarts-268930","workspace_name":"quick-starts-ws-268930","region":"westeurope","compute_target":"compute-cl

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
best_run, fitted_model = automl_run.get_output()
print(best_run)

Run(Experiment: automl-exp,
Id: AutoML_8d854f6c-f80a-439b-abfb-6d9a63b2954b_37,
Type: azureml.scriptrun,
Status: Completed)


In [34]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-exp,AutoML_8d854f6c-f80a-439b-abfb-6d9a63b2954b_37,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [11]:
print(fitted_model)

Pipeline(steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, is_cross_validation=True, working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook268930/code/Users/odl_user_268930')),
                ('prefittedsoftvotingclassifier',
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('11', Pipeli...u'}), reg_alpha=2.1875, reg_lambda=1.0416666666666667, subsample=1, tree_method='auto'))]))], flatten_transform=False, weights=[0.2, 0.06666666666666667, 0.13333333333333333, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667]))])
Y_transformer(['LabelEncoder', LabelEncoder()])


In [12]:
best_run.get_metrics()

{'weighted_accuracy': 0.8818215942356282,
 'precision_score_macro': 0.8784149009734195,
 'average_precision_score_micro': 0.9376396523780757,
 'precision_score_micro': 0.877434283598667,
 'precision_score_weighted': 0.8828568745581666,
 'accuracy': 0.877434283598667,
 'balanced_accuracy': 0.8722667436950994,
 'matthews_correlation': 0.7504702804472837,
 'average_precision_score_macro': 0.9358426292863588,
 'log_loss': 0.3546725904173901,
 'f1_score_macro': 0.8720788777456256,
 'norm_macro_recall': 0.744533487390199,
 'average_precision_score_weighted': 0.9398295580594681,
 'recall_score_micro': 0.877434283598667,
 'AUC_weighted': 0.935154444558243,
 'f1_score_micro': 0.877434283598667,
 'recall_score_weighted': 0.877434283598667,
 'recall_score_macro': 0.8722667436950994,
 'AUC_macro': 0.935154444558243,
 'AUC_micro': 0.9372162304717726,
 'f1_score_weighted': 0.8770305462531344,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_8d854f6c-f80a-439b-abfb-6d9a63b2954b_37/accur

In [13]:
best_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'confusion_matrix',
 'explanation/afccde95/classes.interpret.json',
 'explanation/afccde95/eval_data_viz.interpret.json',
 'explanation/afccde95/expected_values.interpret.json',
 'explanation/afccde95/features.interpret.json',
 'explanation/afccde95/global_names/0.interpret.json',
 'explanation/afccde95/global_rank/0.interpret.json',
 'explanation/afccde95/global_values/0.interpret.json',
 'explanation/afccde95/local_importance_values.interpret.json',
 'explanation/afccde95/per_class_names/0.interpret.json',
 'explanation/afccde95/per_class_rank/0.interpret.json',
 'explanation/afccde95/per_class_values/0.interpret.json',
 'explanation/afccde95/rich_metadata.interpret.json',
 'explanation/afccde95/true_ys_viz.interpret.json',
 'explanation/afccde95/visualization_dict.interpret.json',
 'explanation/afccde95/ys_pred_proba_viz.interpret.json',
 'explanation/afccde95/ys_pred_viz.interpret.json',
 'explanation/f37be7de/classes.interpret.json',
 'expl

In [17]:
best_run.download_file('outputs/conda_env_v_1_0_0.yml','conda_env.yml')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [18]:
#TODO: Save the best model
registered_model = best_run.register_model(model_path="outputs/model.pkl", model_name='best_automl', properties={'accuracy':best_run.get_metrics()["accuracy"]})

In [16]:
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'scoring.py')

In [19]:
from azureml.core import Model
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

environment = best_run.get_environment()
inference_config = InferenceConfig(entry_script = 'scoring.py', environment = environment)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "capstone-best-model", 
                       [registered_model], 
                       inference_config, 
                       deployment_config)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-10-21 10:09:17+00:00 Creating Container Registry if not exists.
2024-10-21 10:09:18+00:00 Use the existing image.
2024-10-21 10:09:20+00:00 Submitting deployment to compute.
2024-10-21 10:09:28+00:00 Checking the status of deployment capstone-best-model..
2024-10-21 10:14:10+00:00 Checking the status of inference endpoint capstone-best-model.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [28]:
import json
# sample two data points from original df to test send request 
test_df = df.sample(2)
label_df = test_df.pop('HeartDisease')

request = json.dumps({'data': test_df.to_dict(orient='records')})
print(request)


{"data": [{"Age": 58, "Sex": "M", "ChestPainType": "ASY", "RestingBP": 120, "Cholesterol": 0, "FastingBS": false, "RestingECG": "LVH", "MaxHR": 106, "ExerciseAngina": true, "Oldpeak": 1.5, "ST_Slope": "Down"}, {"Age": 47, "Sex": "M", "ChestPainType": "NAP", "RestingBP": 140, "Cholesterol": 193, "FastingBS": false, "RestingECG": "Normal", "MaxHR": 145, "ExerciseAngina": true, "Oldpeak": 1.0, "ST_Slope": "Flat"}]}


In [29]:
import requests 

key = service.get_keys()[0]
headers = {'Content-type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'
response = requests.post(service.scoring_uri, request, headers=headers)

In [32]:
print('Expected:', list(label_df))
print('Response:', response.text)

Expected: [True, True]
Response: "{\"result\": [true, true]}"


TODO: In the cell below, print the logs of the web service and delete the service

In [33]:
service.get_logs()

'2024-10-21T10:13:54,666613300+00:00 - rsyslog/run \n2024-10-21T10:13:54,674154200+00:00 - gunicorn/run \n2024-10-21T10:13:54,675963400+00:00 | gunicorn/run | \n2024-10-21T10:13:54,681907300+00:00 | gunicorn/run | ###############################################\n2024-10-21T10:13:54,684151900+00:00 | gunicorn/run | AzureML Container Runtime Information\n2024-10-21T10:13:54,690119300+00:00 | gunicorn/run | ###############################################\n2024-10-21T10:13:54,693222300+00:00 | gunicorn/run | \n2024-10-21T10:13:54,701966000+00:00 | gunicorn/run | \n2024-10-21T10:13:54,703880700+00:00 - nginx/run \n2024-10-21T10:13:54,715203100+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20240918.v1\n2024-10-21T10:13:54,716929500+00:00 | gunicorn/run | \n2024-10-21T10:13:54,722112800+00:00 | gunicorn/run | \n2024-10-21T10:13:54,725608900+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/

In [35]:
service.delete()

Running
2024-10-21 11:37:33+00:00 Check and wait for operation (dde6d109-2bf1-421b-82f1-997a8e5d3bbd) to finish.
2024-10-21 11:37:37+00:00 Deleting service entity.
Succeeded


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
